In [1]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
# from vgg16 import VGG16_LSTM
import pickle
import torch.nn as nn
import torch
import datetime
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
import logging
import torch
import torch.nn as nn
from qx_noface_no2d_video_swin_transformer01 import SwinTransformer3D
import gc
from einops import rearrange
from tqdm import tqdm
batchsz = 4
frame_nums=15
start_epoch = 0
test_flag=False
model_name ="AST_VST——text batchsize4"
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./Logs/personalityLog.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)
USE_CUDA = torch.cuda.is_available()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
logger.debug('============={}+Video swin transformerTrain Strat!============='.format(model_name))
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)

[2025-06-15 23:25:22,820]::1779168656::DEBUG::=============AST_VST——text batchsize4+Video swin transformerTrain Strat!=============


True
학습을 진행하는 기기: cuda


In [3]:
def getdata(pathname,datatype):
    if datatype=="train":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            # train_data_set.extend(pickle.load(output_file))
            final_data_set.extend(pickle.load(output_file))
            # print(i)
        return final_data_set
    elif datatype=="valid":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            valid_data_set=[]
            final_data_set.extend(pickle.load(output_file))
            del valid_data_set
        return final_data_set
print(1)

1


In [4]:
def datasize(type,batchsz):
    if type=='train':
        i=5076#5967
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
    else:
        i=1700#1986
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
print(1)

1


In [5]:
save_model_file_path = '/home/user/10TB/personalityAI/multimodal/KETI2025_finetuned_videoswintransformer/{}_{}.{}'
#mtcnn= MTCNN(image_size=224, margin=0, min_face_size= 20, 
#             thresholds=[0.6, 0.7, 0.7], post_process=True)   # 안씀
print(1)

1


#### dataset 구성

0 : audio   
1 : fullshot video   
2 : face   
3 : lip   
4 : text   
5 : tag   

In [6]:
def reshape_to_expected_traininput(dataset: List[Tuple[ np.ndarray, np.ndarray]]) -> Tuple[
     np.ndarray, np.ndarray]:
    result=[]
    for i in range(0, len(dataset[0])):
        result.append((dataset[0][i],dataset[1][i],dataset[2][i],dataset[3][i]))
    result = result[:datasize("train",batchsz)]
    x0_list = []
    x1_list = []
    x2_list = []
    x3_list = []

    for i in range(0, len(result)):
        x0_list.append(result[i][0])
        x1_list.append(result[i][1])
        x2_list.append(result[i][2])
        x3_list.append(result[i][3])

    return (np.stack(x0_list), np.stack(x1_list), np.stack(x2_list),np.stack(x3_list))
print(1)

1


In [7]:
def reshape_to_expected_validinput(dataset: List[Tuple[ np.ndarray, np.ndarray]]) -> Tuple[
     np.ndarray, np.ndarray]:
    result=[]
    for i in range(0, len(dataset[0])):
        result.append((dataset[0][i],dataset[1][i],dataset[2][i],dataset[3][i]))
    result = result[:datasize("valid",batchsz)]
    x0_list = []
    x1_list = []
    x2_list = []
    x3_list = []

    for i in range(0, len(result)):
        x0_list.append(result[i][0])
        x1_list.append(result[i][1])
        x2_list.append(result[i][2])
        x3_list.append(result[i][3])

    return (np.stack(x0_list), np.stack(x1_list), np.stack(x2_list),np.stack(x3_list))
print(1)

1


In [8]:
validation_set_data=getdata("Your Path","valid")
print(len(validation_set_data))
#print(validation_set_data)

validation_set_data=reshape_to_expected_validinput(validation_set_data)
print(1)

4
1


In [9]:
def getdata2(pathname,datatype):
    if datatype=="train":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            # train_data_set.extend(pickle.load(output_file))
            final_data_set.extend(pickle.load(output_file))
            # print(len(pickle.load(output_file)))
        return final_data_set

In [10]:
#test_set_data=getdata2("/home/user/10TB/Data/KETI2025/김봉재교수님연구실/full_video/15frame/scene1_","train")
#test_set_data=reshape_to_expected_traininput(test_set_data)
#print(1)

In [11]:
# """
# 0 : audio
# 1 : fullshot video
# 2 : face
# 3 : lip
# 4 : text
# 5 : tag
# """


# print("Audio: ", test_set_data[0].shape, "\n",
#      "fullshot video: ", test_set_data[1].shape, "\n",
#       "face: ", test_set_data[2].shape, "\n",
#       "lip: ", test_set_data[3].shape, "\n",
#       "text: ", test_set_data[4].shape, "\n",
#       "tag: ", test_set_data[5].shape, "\n")

In [12]:
train_set_data=getdata("Your Path","train")
train_set_data=reshape_to_expected_traininput(train_set_data)
print(1)

1


In [13]:
validation_set_data[0]  # 6 x 1692

array([[[[ 0.        ],
         [ 0.        ],
         [ 0.        ],
         ...,
         [-4.7925735 ],
         [-4.7881193 ],
         [-4.9050574 ]],

        [[ 0.        ],
         [ 0.        ],
         [ 0.        ],
         ...,
         [ 0.7451298 ],
         [ 0.74512583],
         [ 0.8219356 ]],

        [[ 0.        ],
         [ 0.        ],
         [ 0.        ],
         ...,
         [ 0.13816968],
         [ 0.12655732],
         [ 0.17270342]],

        ...,

        [[ 0.        ],
         [ 0.        ],
         [ 0.        ],
         ...,
         [ 0.14503907],
         [ 0.13050982],
         [ 0.12636714]],

        [[ 0.        ],
         [ 0.        ],
         [ 0.        ],
         ...,
         [ 0.13912871],
         [ 0.13201621],
         [ 0.1432339 ]],

        [[ 0.        ],
         [ 0.        ],
         [ 0.        ],
         ...,
         [ 0.07743692],
         [ 0.07799025],
         [ 0.08155744]]],


       [[[ 0.        ],


In [14]:
print("Audio: ", validation_set_data[0].shape, "\n",
     "video: ", validation_set_data[1].shape, "\n",
      "text: ", validation_set_data[2].shape, "\n",
      "labels: ", validation_set_data[3].shape, "\n")

Audio:  (1700, 24, 6000, 1) 
 video:  (1700, 15, 224, 224, 3) 
 text:  (1700,) 
 labels:  (1700, 5, 1) 



In [15]:
print("Audio: ", train_set_data[0].shape, "\n",
     "video: ", train_set_data[1].shape, "\n",
      "text: ", train_set_data[2].shape, "\n",
      "labels: ", train_set_data[3].shape, "\n")

Audio:  (5076, 24, 6000, 1) 
 video:  (5076, 15, 224, 224, 3) 
 text:  (5076,) 
 labels:  (5076, 5, 1) 



In [16]:
validation_set_data[3][634]

array([[92.],
       [61.],
       [77.],
       [67.],
       [71.]])

In [17]:
validation_set_data[3][6]

array([[81.],
       [78.],
       [59.],
       [92.],
       [84.]])

In [18]:
len(train_set_data)  # 6 x 5052

4

In [19]:
def imagecrop(imagedata,batchsz):  # 안씀
    output=[]
    for j in range(batchsz):#비디오 데이터 batchsz(9) 개 있다
        firstoutput=[]
        for i in range(len(imagedata[j])):#imagedata에 15개의 프레임이 있다
            temp = imagedata[j][i].unfold(1,112,112).unfold(2,112,112)
            temp = rearrange(temp, 'c h1 w1 h w -> (h1 w1) c h w')
            firstoutput.append(temp)
        firstoutput = torch.stack(firstoutput,0)
        output.append(firstoutput)
    output = torch.stack(output,0)
    return output
print(1)

1


In [20]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])
print(1)

1


In [21]:
# 데이터 텐서로 만들기

class ChalearnDataset(Dataset):  
    def __init__(self,audiodata,fullshot,textdata,tagdata,transform=None):
        self.audiodata=audiodata
        self.fullshot=fullshot
        self.tagdata=tagdata
        self.textdata=textdata
        self.transform = transform  # 표준화 여부
    def __len__(self):
        return len(self.fullshot)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()#텐서의 경우 목록으로 돌아가기
        audiodata = self.audiodata[idx]
        audiodata = torch.FloatTensor(audiodata)
        fullshot=self.fullshot[idx]
        fullshot=torch.FloatTensor(fullshot)
        textdata = self.textdata[idx]
        # image_data=image_data.reshape(15,3,224,224)
        big_five_sorces=self.tagdata[idx]
        big_five_sorces = torch.FloatTensor((big_five_sorces-24)/(120-24))
        return audiodata,fullshot,textdata,big_five_sorces
print(1)

1


In [22]:
num_workerssz = 12
lr = 1e-04
epochs = 20  #  120
print(1)

1


In [23]:
train_set_data = ChalearnDataset(audiodata=train_set_data[0],
                                 fullshot = train_set_data[1],
                                 textdata=train_set_data[2],
                                 tagdata=train_set_data[3],
                                 transform=transform)
val_set_data = ChalearnDataset(audiodata=validation_set_data[0],
                               fullshot = validation_set_data[1],
                               textdata=validation_set_data[2],
                               tagdata=validation_set_data[3],
                               transform=transform)
# test_set_data = ChalearnDataset(audiodata=test_set_data[0],
#                                fullshot = test_set_data[1],
#                                textdata=test_set_data[2],
#                                tagdata=test_set_data[3],
#                                transform=transform)

train_dataloader = DataLoader(dataset=train_set_data, batch_size=batchsz, 
                              shuffle=True, num_workers=num_workerssz)
val_dataloader = DataLoader(dataset=val_set_data, batch_size=batchsz, 
                            shuffle=True, num_workers=num_workerssz)
# test_dataloader = DataLoader(dataset=test_set_data, batch_size=batchsz, 
#                             shuffle=True, num_workers=num_workerssz)
max_value=0
print(1)

1


In [24]:
val_set_data.__getitem__(50)[3].size()

torch.Size([5, 1])

In [ ]:
model=SwinTransformer3D(fusionlayer=0)
model.to(device)
criterion = torch.nn.L1Loss().to(device)  # 손실함수
optimizer = torch.optim.AdamW(model.parameters(), lr=lr) # 옵티마이저

if test_flag==True:
    checkpoint=torch.load(save_model_file_path.format('model',start_epoch,'pth'), map_location=device)
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    # criterion.load_state_dict(checkpoint["loss"])
train_avg_loss0=[]
val_avg_loss0=[]
print(1)

In [ ]:
from torchinfo import summary

summary(model)

In [ ]:
def extract_ocean(data, batch_size):
    o=[]
    c=[]
    e=[]
    a=[]
    n=[]
    
    for i in range(batch_size):
        try:
            o.append(data[i][0].item())
            c.append(data[i][1].item())
            e.append(data[i][2].item())
            a.append(data[i][3].item())
            n.append(data[i][4].item())
            
        except Exception as err:
            print(err)
            pass
        
    o = torch.tensor(o) 
    c = torch.tensor(c)
    e = torch.tensor(e) 
    a = torch.tensor(a) 
    n = torch.tensor(n) 
    return o,c,e,a,n

In [ ]:
results = []
with torch.cuda.device(0):
    for i in range(start_epoch, epochs):
        train_avg_loss = 0
        val_avg_loss = 0
        train_avg_o_loss = 0
        train_avg_c_loss = 0
        train_avg_e_loss = 0
        train_avg_a_loss = 0
        train_avg_n_loss = 0
        val_avg_o_loss = 0
        val_avg_c_loss = 0
        val_avg_e_loss = 0
        val_avg_a_loss = 0
        val_avg_n_loss = 0
        
        for audiodata, fullshot, textdata, big_five_data in tqdm(train_dataloader):
            big_five_data=big_five_data.permute(0,2,1)
            big_five_data=big_five_data.squeeze()
            audiodata = audiodata.to(device)
            fullshot = fullshot.to(device)
            big_five_data = big_five_data.to(device)
            optimizer.zero_grad()  # 기울기가 0이 됩니다.
            hypothesis = model(fullshot,textdata,audiodata)  # 모델의 예측 결과를 저장합니다.
            loss = criterion(hypothesis, big_five_data)  # 예측된 결과와 실제 태그 사이의 손실 값을 저장합니다.
            loss.backward()  # 역방향 전파입니다. 
            optimizer.step()  
            train_avg_loss += loss  
        train_avg_loss=train_avg_loss/len(train_dataloader)
        
        with torch.no_grad(): #validate
            for audiodata,fullshot,textdata,big_five_data in tqdm(val_dataloader):
                big_five_data=big_five_data.permute(0,2,1)
                big_five_data=big_five_data.squeeze()
                audiodata = audiodata.to(device)
                fullshot = fullshot.to(device)
                big_five_data = big_five_data.to(device)
                hypothesis = model(fullshot,textdata,audiodata)
                val_loss = criterion(hypothesis, big_five_data)
                hypothesiso,hypothesisc,hypothesise,hypothesisa,hypothesisn=extract_ocean(hypothesis,batchsz)
                big_five_datao,big_five_datac,big_five_datae,big_five_dataa,big_five_datan=extract_ocean(big_five_data,batchsz)
                val_o_loss = criterion(hypothesiso, big_five_datao) 
                val_c_loss = criterion(hypothesisc, big_five_datac) 
                val_e_loss = criterion(hypothesise, big_five_datae) 
                val_a_loss = criterion(hypothesisa, big_five_dataa) 
                val_n_loss = criterion(hypothesisn, big_five_datan)  
                val_avg_loss += val_loss
                val_avg_o_loss += val_o_loss
                val_avg_c_loss += val_c_loss
                val_avg_e_loss += val_e_loss
                val_avg_a_loss += val_a_loss
                val_avg_n_loss += val_n_loss
            val_avg_loss=val_avg_loss/len(val_dataloader)
            val_avg_o_loss=val_avg_o_loss/len(val_dataloader)
            val_avg_c_loss=val_avg_c_loss/len(val_dataloader)
            val_avg_e_loss=val_avg_e_loss/len(val_dataloader)
            val_avg_a_loss=val_avg_a_loss/len(val_dataloader)
            val_avg_n_loss=val_avg_n_loss/len(val_dataloader)
        torch.cuda.empty_cache()
        # if (1-val_avg_loss)>=(1-max_value):
        max_value=val_avg_loss
        start_epoch+=1
 #       if (i + 1) % 5 == 0:
        torch.save({
                'epoch': i+1,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'loss': val_avg_loss,
            }, save_model_file_path.format('model(only_image)',start_epoch,'pth'))
        results.append([i+1, train_avg_loss, val_avg_loss, 1-val_avg_loss, 1-val_avg_o_loss, 1-val_avg_c_loss, 1-val_avg_e_loss, 1-val_avg_a_loss,1-val_avg_n_loss])
        logger.debug('\n Epoch: {}, \n train_loss: {:.4f}, \n val_loss: {:.4f}, \n 1-MAE: {:.4f} , \n 1-val_avg_o_loss:{:.4f} , \n 1-val_avg_c_loss:{:.4f} , \n 1-val_avg_e_loss:{:.4f} , \n 1-val_avg_a_loss:{:.4f} , \n 1-val_avg_n_loss:{:.4f} \n'.format(i+1, train_avg_loss, val_avg_loss, 1-val_avg_loss, 1-val_avg_o_loss, 1-val_avg_c_loss, 1-val_avg_e_loss, 1-val_avg_a_loss,1-val_avg_n_loss))
    

### Test

In [ ]:
test_model = SwinTransformer3D()
state_dict_m = torch.load("/home/user/10TB/personalityAI/multimodal/KETI2025_finetuned_videoswintransformer/model_14.pth")

with torch.cuda.device(0):
    test_model.load_state_dict(state_dict_m['model'])
    test_model.to(device)
    test_model.eval()
    
    test_avg_loss = 0
    test_avg_o_loss = 0
    test_avg_c_loss = 0
    test_avg_e_loss = 0
    test_avg_a_loss = 0
    test_avg_n_loss = 0
    
    with torch.no_grad():#validate
        for audiodata,fullshot,textdata,big_five_data in tqdm(test_dataloader):
            big_five_data=big_five_data.permute(0,2,1)
            big_five_data=big_five_data.squeeze()
            audiodata = audiodata.to(device)
            fullshot = fullshot.to(device)
            big_five_data = big_five_data.to(device)
            hypothesis = test_model(fullshot,textdata,audiodata)
            test_loss = criterion(hypothesis, big_five_data)
            hypothesiso,hypothesisc,hypothesise,hypothesisa,hypothesisn=extract_ocean(hypothesis,batchsz)
            big_five_datao,big_five_datac,big_five_datae,big_five_dataa,big_five_datan=extract_ocean(big_five_data,batchsz)
            test_o_loss = criterion(hypothesiso, big_five_datao) 
            test_c_loss = criterion(hypothesisc, big_five_datac) 
            test_e_loss = criterion(hypothesise, big_five_datae) 
            test_a_loss = criterion(hypothesisa, big_five_dataa) 
            test_n_loss = criterion(hypothesisn, big_five_datan)  
            
            test_avg_loss += test_loss
            test_avg_o_loss += test_o_loss
            test_avg_c_loss += test_c_loss
            test_avg_e_loss += test_e_loss
            test_avg_a_loss += test_a_loss
            test_avg_n_loss += test_n_loss
            
        test_avg_loss = test_avg_loss / len(test_dataloader)
        test_avg_o_loss = test_avg_o_loss / len(test_dataloader)
        test_avg_c_loss = test_avg_c_loss / len(test_dataloader)
        test_avg_e_loss = test_avg_e_loss / len(test_dataloader)
        test_avg_a_loss = test_avg_a_loss / len(test_dataloader)
        test_avg_n_loss = test_avg_n_loss / len(test_dataloader)
        
    torch.cuda.empty_cache()
    # if (1-val_avg_loss)>=(1-max_value):
    print('1-MAE: {:.4f} , 1-o_loss:{:.4f} , 1-c_loss:{:.4f} , 1-e_loss:{:.4f} , 1-a_loss:{:.4f} , 1-n_loss:{:.4f}'.format(1-test_avg_loss, 1-test_avg_o_loss, 1-test_avg_c_loss, 1-test_avg_e_loss, 1-test_avg_a_loss,1-test_avg_n_loss))